In [1]:

import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import pandas as pd
import math
from numpy import diff
from numpy import gradient
import numpy as np
from scipy import pi
from scipy import fft
from scipy.fft import fft, fftfreq
from scipy import signal
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots


In [2]:
from gps import get_data

In [3]:
df1=pd.read_csv('mtb_phd/minerals/OPPOFindX5Pro-03-15-02/OPPOFindX5Pro-03-15-02.miq', 
                skiprows=57, skipfooter=35,names=['sample', 'Lat','Lon', 'Alt','Time', 'Speed'])

#postion,sensors  = get_data(mtb_phd/minerals/OPPOFindX5Pro-03-15-02/OPPOFindX5Pro-03-15-02)
df=pd.read_csv('mtb_phd/drive-download-20230823T084905Z-001/BrakeAce 20230315180854 Minerals +Torque.csv') #,usecols=[1,2,3,4,5], 
               #names=( 'fork','rearWheel','frontForce','shock','rearForce'), header=0)
               
df

/tmp/ipykernel_8325/1410175824.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df1=pd.read_csv('mtb_phd/minerals/OPPOFindX5Pro-03-15-02/OPPOFindX5Pro-03-15-02.miq',


,Time (s),Distance (m),Latitude (deg),Longitude (deg),Elevation (m),Speed (km/h),Front Brake Power (W),Rear Brake Power (W),Front Brake Torque (Nm),Rear Brake Torque (Nm)
0,38.2,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
1,38.3,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
2,38.4,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
3,38.5,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
4,38.6,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2302,268.4,1106,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
2303,268.5,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
2304,268.6,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
2305,268.7,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0


In [27]:
df.columns

Index(['Time (s)', ' Distance (m)', ' Latitude (deg)', ' Longitude (deg)',
       ' Elevation (m)', ' Speed (km/h)', ' Front Brake Power (W)',
       ' Rear Brake Power (W)', ' Front Brake Torque (Nm)',
       ' Rear Brake Torque (Nm)'],
      dtype='object')

In [4]:
    # size=len(df1.index)
    # timedata,interval=np.linspace(0,size/200,num=size,retstep=True, endpoint=False) #create timestamps
    # df1['time']=np.round(timedata,3) # add to dataframe, round to 0.001s
df1['elapsed_time'] = df1['Time'] - df1['Time'][0]

In [5]:
df1

,sample,Lat,Lon,Alt,Time,Speed,elapsed_time
0,3,-38.170776,176.302207,634.598145,1.678857e+09,2.345496,0.000
1,4,-38.170746,176.302225,634.472534,1.678857e+09,2.939015,1.195
2,5,-38.170705,176.302245,634.472534,1.678857e+09,2.838630,1.792
3,6,-38.170700,176.302249,633.794678,1.678857e+09,2.679965,2.298
4,7,-38.170688,176.302255,627.541016,1.678857e+09,2.634642,2.867
...,...,...,...,...,...,...,...
293,296,-38.166518,176.308504,454.841675,1.678857e+09,0.193711,233.645
294,297,-38.166518,176.308506,455.514648,1.678857e+09,0.135244,234.242
295,298,-38.166518,176.308508,455.976929,1.678857e+09,0.086141,234.839
296,299,-38.166517,176.308509,455.976929,1.678857e+09,0.062891,235.342


In [76]:
a = np.array([df1['elapsed_time'],df1['Speed']])
b = np.array([df['Time (s)'],df[' Speed (km/h)']])
# a = np.array(df1['Speed'])
# b = np.array(df[' Speed (km/h)'])
print(a)
print(b)

[[0.00000000e+00 1.19500017e+00 1.79200006e+00 2.29800010e+00
  2.86700010e+00 3.45700002e+00 4.03400016e+00 4.53900003e+00
  5.13600016e+00 5.73300004e+00 6.27000022e+00 7.27200007e+00
  7.85200000e+00 9.04700017e+00 9.63900018e+00 1.02720001e+01
  1.12720001e+01 1.22710001e+01 1.32700002e+01 1.42720001e+01
  1.52700002e+01 1.58350000e+01 1.64310002e+01 1.70080001e+01
  1.75580001e+01 1.82720001e+01 1.92710001e+01 2.02720001e+01
  2.12720001e+01 2.22720001e+01 2.32710001e+01 2.38460002e+01
  2.44430001e+01 2.50400002e+01 2.56370001e+01 2.62340002e+01
  2.68300002e+01 2.74280002e+01 2.82710001e+01 2.92720001e+01
  3.02700002e+01 3.08320000e+01 3.12710001e+01 3.22720001e+01
  3.32720001e+01 3.42710001e+01 3.52720001e+01 3.62710001e+01
  3.72710001e+01 3.82720001e+01 3.92710001e+01 4.02710001e+01
  4.12710001e+01 4.22720001e+01 4.32710001e+01 4.42710001e+01
  4.52720001e+01 4.62710001e+01 4.72710001e+01 4.82700002e+01
  4.92720001e+01 5.02700002e+01 5.12710001e+01 5.22700002e+01
  5.3271

In [66]:
help(fastdtw)

Help on built-in function fastdtw in module fastdtw._fastdtw:

fastdtw(...)
    return the approximate distance between 2 time series with O(N)
    time and memory complexity
    
    Parameters
    ----------
    x : array_like
        input array 1
    y : array_like
        input array 2
    radius : int
        size of neighborhood when expanding the path. A higher value will
        increase the accuracy of the calculation but also increase time
        and memory consumption. A radius equal to the size of x and y will
        yield an exact dynamic time warping calculation.
    dist : function or int
        The method for calculating the distance between x[i] and y[j]. If
        dist is an int of value p > 0, then the p-norm will be used. If
        dist is a function then dist(x[i], y[j]) will be used. If dist is
        None then abs(x[i] - y[j]) will be used.
    
    Returns
    -------
    distance : float
        the approximate distance between the 2 time series
    path

In [77]:
fig=px.line(x=a[0], y=a[1])
fig.add_trace(go.Scatter(x=b[0], y=b[1]))

fig.show()

In [78]:
distance, path = fastdtw(a.T, b.T, dist=euclidean) #radius=100, 
print(distance)

9840.896045661217


In [75]:
print(path)

[(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10), (12, 11), (13, 12), (14, 13), (15, 14), (16, 15), (17, 16), (18, 17), (19, 18), (20, 19), (21, 20), (22, 21), (23, 22), (24, 23), (25, 24), (26, 25), (27, 26), (28, 27), (29, 28), (30, 29), (31, 30), (32, 31), (33, 32), (34, 33), (35, 34), (36, 35), (37, 36), (38, 37), (39, 38), (40, 39), (41, 40), (42, 41), (43, 42), (44, 43), (45, 44), (46, 45), (47, 46), (48, 47), (49, 48), (50, 49), (51, 50), (52, 51), (53, 52), (54, 53), (55, 54), (56, 55), (56, 56), (56, 57), (56, 58), (56, 59), (56, 60), (56, 61), (56, 62), (56, 63), (56, 64), (56, 65), (56, 66), (56, 67), (56, 68), (56, 69), (56, 70), (56, 71), (56, 72), (56, 73), (56, 74), (56, 75), (56, 76), (56, 77), (56, 78), (56, 79), (56, 80), (56, 81), (56, 82), (56, 83), (56, 84), (56, 85), (56, 86), (56, 87), (56, 88), (56, 89), (56, 90), (56, 91), (56, 92), (57, 93), (58, 94), (59, 95), (59, 96), (59, 97), (59, 98), (59, 99), (59, 100),

In [81]:

shifted=[]
shiftedb=[]
for m in path:
    d=m[0]
    e=m[1]
    shifted.append(d)
    shiftedb.append(e)
    



In [85]:
set1=a[1][shifted]
set2=b[1][shiftedb]

In [87]:


fig=px.line( y=set1)
fig.add_trace(go.Scatter(y=set2))

fig.show()